In [1]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

# Data Load

In [2]:
sub_path = "data/subs"
all_files = os.listdir(sub_path)
all_files

['blends',
 'different-models-for-types3.csv',
 'mpnn-ensemble-sagpool.csv',
 'sagpooling_larger_mpnn.csv',
 'semgcn_1c_w12_24k.csv',
 'submission_per_type_inflated.csv',
 'submission_type_important_features.csv',
 'submit-00325000_model-larger.csv',
 'submit-ensemble-JHC-all_types3.csv',
 'sub_qm9_babel_ascf_11fold_extreme_AND_stack_median_AVG.csv']

In [3]:
selected_files = ['submit-ensemble-JHC-all_types3.csv', 'sub_qm9_babel_ascf_11fold_extreme_AND_stack_median_AVG.csv',
                  'semgcn_1c_w12_24k.csv', 'sagpooling_larger_mpnn.csv', ]

In [7]:
# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in selected_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "champ" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
ncol = concat_sub.shape[1]
concat_sub.head()

/home/christian/anaconda3/envs/machine_learning_general/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,id,champ0,champ1,champ2,champ3
0,4658147,8.040669,15.893167,12.576186,9.848858
1,4658148,127.524742,194.245589,155.951752,178.970093
2,4658149,2.342010,7.173190,3.237241,2.411096
3,4658150,127.524742,189.553700,155.951767,178.970093
4,4658151,8.040669,15.460785,12.576186,9.848858


In [8]:
# check correlation
concat_sub.iloc[:,1:ncol].corr()

,champ0,champ1,champ2,champ3
champ0,1.000000,0.999901,0.999926,0.999937
champ1,0.999901,1.000000,0.999913,0.999889
champ2,0.999926,0.999913,1.000000,0.999916
champ3,0.999937,0.999889,0.999916,1.000000


In [9]:
# get the data fields ready for stacking
concat_sub['champ_max'] = concat_sub.iloc[:, 1:ncol].max(axis=1)
concat_sub['champ_min'] = concat_sub.iloc[:, 1:ncol].min(axis=1)
concat_sub['champ_mean'] = concat_sub.iloc[:, 1:ncol].mean(axis=1)
concat_sub['champ_median'] = concat_sub.iloc[:, 1:ncol].median(axis=1)

In [10]:
concat_sub.iloc[:, 1:ncol].describe()

,champ0,champ1,champ2,champ3
count,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06
mean,1.587813e+01,1.588382e+01,1.589216e+01,1.586508e+01
std,3.485898e+01,3.486016e+01,3.486960e+01,3.480879e+01
min,-4.177213e+01,-3.208316e+01,-3.278223e+01,-3.660491e+01
25%,-2.393832e-01,-2.148092e-01,-2.406806e-01,-2.448642e-01
50%,2.289008e+00,2.277482e+00,2.277586e+00,2.284614e+00
75%,7.327051e+00,7.316506e+00,7.345392e+00,7.351202e+00
max,2.043061e+02,2.032502e+02,2.071149e+02,2.035825e+02


In [11]:
cutoff_lo = -35
cutoff_hi = 207

# Mean Stacking

In [15]:
concat_sub['scalar_coupling_constant'] = concat_sub['champ_mean']
concat_sub[['id', 'scalar_coupling_constant']].to_csv('data/subs/blends/stack_mean.csv', index=False)

**LB----**

# Median Stacking

In [12]:
concat_sub['scalar_coupling_constant'] = concat_sub['champ_median']
concat_sub[['id', 'scalar_coupling_constant']].to_csv('data/subs/blends/stack_median_lgbm_mpnn_semgcn.csv', index=False)

**LB -1.901**

# PushOut + Median Stacking 

Pushout strategy is a bit agressive given what it does...

In [11]:
concat_sub['scalar_coupling_constant'] = np.where(np.all(concat_sub.iloc[:,1:ncol] > cutoff_lo, axis=1), 1, 
                                    np.where(np.all(concat_sub.iloc[:,1:ncol] < cutoff_hi, axis=1),
                                             0, concat_sub['champ_median']))
concat_sub[['id', 'scalar_coupling_constant']].to_csv('data/subs/blends/stack_pushout_median.csv', index=False)

> **LB -----**

# MinMax + Mean Stacking

MinMax seems more gentle and it outperforms the previous one given its peformance score.

In [12]:
concat_sub['scalar_coupling_constant'] = np.where(np.all(concat_sub.iloc[:,1:ncol] > cutoff_lo, axis=1), 
                                    concat_sub['champ_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:ncol] < cutoff_hi, axis=1),
                                             concat_sub['champ_min'], 
                                             concat_sub['champ_mean']))
concat_sub[['id', 'scalar_coupling_constant']].to_csv('data/subs/blends/stack_minmax_mean.csv', index=False)

> **LB ----**



# MinMax + Median Stacking 

In [14]:
concat_sub['scalar_coupling_constant'] = np.where(np.all(concat_sub.iloc[:,1:ncol] > cutoff_lo, axis=1), 
                                    concat_sub['champ_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:ncol] < cutoff_hi, axis=1),
                                             concat_sub['champ_min'], 
                                             concat_sub['champ_median']))
concat_sub[['id', 'scalar_coupling_constant']].to_csv('data/subs/blends/stack_minmax_median.csv', index=False)

**LB ----** -

# MinMax + BestBase Stacking

In [14]:
# load the model with best base performance
sub_base = pd.read_csv('data/subs/submission_type_important_features.csv')

In [17]:
concat_sub['champ_base'] = sub_base['scalar_coupling_constant']
concat_sub['id'] = sub_base['id']
concat_sub['scalar_coupling_constant'] = np.where(np.all(concat_sub.iloc[:,1:ncol] > cutoff_lo, axis=1), 
                                    concat_sub['champ_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:ncol] < cutoff_hi, axis=1),
                                             concat_sub['champ_min'], 
                                             concat_sub['champ_base']))
concat_sub[['id', 'scalar_coupling_constant']].to_csv('data/subs/blends/stack_minmax_bestbase.csv', index=False)

> **LB----** -

### Weighted Average

In [62]:
selected_files = ['sub_qm9_babel_ascf_11fold_extreme_AND_stack_median_AVG.csv',
                  'semgcn_1c_w12_24k.csv', 'sagpooling_larger_mpnn.csv', ]

In [63]:
sub_lgbm = pd.read_csv(os.path.join(sub_path, selected_files[0]), index_col=0)
sub_semgcn = pd.read_csv(os.path.join(sub_path, selected_files[1]), index_col=0)
sub_mpnn = pd.read_csv(os.path.join(sub_path, selected_files[2]), index_col=0)

In [69]:
cc = sub_semgcn.copy()

In [76]:
cc['scalar_coupling_constant'] = (0.2*sub_lgbm['scalar_coupling_constant'] + 0.5*sub_semgcn['scalar_coupling_constant']
                                + 0.3*sub_mpnn['scalar_coupling_constant'])

In [77]:
print(cc['scalar_coupling_constant'].mean()-sub_lgbm['scalar_coupling_constant'].mean())
print(cc['scalar_coupling_constant'].mean()-sub_semgcn['scalar_coupling_constant'].mean())
print(cc['scalar_coupling_constant'].mean()-sub_mpnn['scalar_coupling_constant'].mean())

-0.0014508007425977354
-0.00979137145056086
0.0172861529099535


In [79]:
cc.to_csv('weighted_average_lgbm_semgcn_mpnn.csv')

In [80]:
cc.head()

,scalar_coupling_constant
id,
4658147,12.421384
4658148,170.516021
4658149,3.776587
4658150,169.577651
4658151,12.334907
